In [1]:
from __future__ import division
from __future__ import print_function

import sys
import os
import argparse
import time
import random
from glob import glob

import align.detect_face
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import misc
from scipy.interpolate import Rbf
from skimage import io, transform
from six.moves import xrange

import tensorflow as tf
import tensorflow.contrib.slim as slim

import facenet
import helper

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

plt.ion() # interactive mode

In [2]:
landmarks_frame = pd.read_csv('./face_landmarks_generate.csv')
file_list = landmarks_frame.image_name.values.tolist()

In [5]:
batch_size = 100

In [27]:
avgP_container = np.load('f_avgP_list.npz')
emb_container = np.load('f_emb_list.npz')

train_set = []
test_set = []

for key in sorted(emb_container, key=lambda x: int(x.strip('arr_'))) :
    batch = avgP_container[key], emb_container[key]
    if len(batch[0]) is batch_size :
        train_set.append(batch)
    else :
        test_set.append(batch)

test_set_index = len(train_set)

In [7]:
t_dataset = helper.Dataset('nf',file_list, 160)
print(t_dataset.shape)

(7867, 160, 160, 3)


In [28]:
print(len(train_set))

78


In [43]:
def F_layer(encoded, f_num = 128) : 
    with tf.variable_scope('F') :
        fc = slim.fully_connected(encoded, f_num, activation_fn=tf.nn.relu, scope='fc')
    return fc

In [44]:
def MLP(net, landmark_num = 68, reuse=None, scope='MLP'):
    """Builds the MLP for landmark"""
    with tf.variable_scope(scope, 'MLP') :
        net = slim.fully_connected(net, 256, activation_fn=None, scope='fc0')
        net = slim.fully_connected(net, 128, activation_fn=None, scope='fc1')
        net = slim.fully_connected(net, landmark_num, activation_fn=tf.nn.relu, scope='fc2')
    return net

In [45]:
def landmark_decode(net, landmark_num = 68):
    with tf.variable_scope('landmark') :
        decoded_x = MLP(net, scope= 'decoded_x')
        decoded_y = MLP(net, scope= 'decoded_y')
    return decoded_x, decoded_y

In [46]:
def CNN(F, size) :
    with tf.variable_scope('CNN') :
        # 12 x 12 x 256
        f_size = int(size / 8)
        features = slim.fully_connected(F, f_size * f_size * 256, activation_fn=None, scope="features")
        features = tf.reshape(features, [-1, f_size, f_size, 256])
        # print(features.shape)
        
        # 24 x 24 x 128
        upsample_0 = slim.conv2d_transpose(features, 128, 5, stride=2, scope="upsample_0")
        # print(upsample_0.shape)
        
        # 48 x 48 x 64
        upsample_1 = slim.conv2d_transpose(upsample_0, 64, 5, stride=2, scope="upsample_1")
        # print(upsample_1.shape)
        
        # 96 x 96 x 32
        upsample_2 = slim.conv2d_transpose(upsample_1, 32, 5, stride=2, scope="upsample_2")
        # print(upsample_2.shape)
        
        # 96 x 96 x 3
        one_by_one_conv = slim.conv2d(upsample_2, 3, 1, stride=1, activation_fn=None, scope="one_by_one_conv")
        # print(one_by_one_conv.shape)
    return one_by_one_conv

In [47]:
def texture_decode(encoded, size) :
    with tf.variable_scope('texture') :
        cnn = CNN(F, size)
    return cnn

In [48]:
def get_grids(size):
    return np.mgrid[0:size-1:(size * 1j), 0:size-1:(size * 1j)]

In [49]:
def get_zero_displacement(size, landmarks):
    mid = size/2
    end = size-1
    
    new_landmarks = np.copy(landmarks)
    new_landmarks[:, 0][27:31] += 5
#     new_landmarks[:, 1][27:31] += 5
    
    zero_displacement = [[0,0], 
                         [0, mid], 
                         [0, end], 
                         [mid, 0], 
                         [end,0], 
                         [end, mid], 
                         [end, end], 
                         [mid, end]]
    
    landmarks_with_zero_displacement = np.append(landmarks, zero_displacement, axis=0)
    new_landmarks_with_zero_displacement = np.append(new_landmarks, zero_displacement, axis=0)
    return landmarks_with_zero_displacement, new_landmarks_with_zero_displacement

In [50]:
def rbf_tf(pred_x, pred_y, correct_points, grids, grid_shape, batch_size):
    def _euclidean_norm_tf(x1, x2):    
        return tf.sqrt(tf.reduce_sum(((x1 - x2)**2), 1))

    def _h_linear_tf(r):
        return r

    def _call_norm_tf(x1, x2):
        x1 = tf.expand_dims(x1, 3) 
        x2 = tf.expand_dims(x2, 2) 
        return norm(x1, x2)    

    # set parameters
    norm = _euclidean_norm_tf
    basis_function = _h_linear_tf
    epsilon = tf.constant(2.)
    smooth = tf.constant(1.)

    xi = tf.concat([tf.expand_dims(pred_x, 1), tf.expand_dims(pred_y, 1)], 1) # (None, 2, 76)
    N = xi.shape[-1].value # same as landmarks_num => 76
    di = tf.expand_dims(correct_points, 2) # (None, 76, 1)
    
    r = _call_norm_tf(xi, xi) # (None, 76, 76)
    
    batch_shape = tf.shape(pred_x)[0:1]
    A = tf.subtract(basis_function(r), tf.multiply(smooth, tf.eye(N, batch_shape= batch_shape)))
    #A = tf.subtract(basis_function(r), tf.multiply(smooth, tf.eye(N, batch_shape= [batch_size])))
    nodes = tf.matrix_solve (A, di)
    r2 = _call_norm_tf(grids, xi)
    return tf.reshape(tf.matmul(r2, nodes), [-1, grid_shape[0], grid_shape[1]])

In [51]:
def warp_tf(data, pred_x, pred_y, correct_x, correct_y, grids, grid_shape, batch_size) :
    rbf_x = rbf_tf(pred_x, pred_y, correct_x, grids, grid_shape, batch_size)
    rbf_y = rbf_tf(pred_x, pred_y, correct_y, grids, grid_shape, batch_size)
    warp = tf.concat([tf.expand_dims(rbf_x, 3), tf.expand_dims(rbf_y, 3)], axis=3)
    resample = tf.contrib.resampler.resampler(data=data, warp=warp)
    return resample

In [88]:
epochs = 200

avgP_num = 1792
emb_num = 128
f_num = 512 #1024

l_num = 68
zd_l_num = 74
t_size = 160
t_channel = 3

grid_y, grid_x = get_grids(t_size)
grid_shape = grid_x.shape
xa = np.asarray([a.flatten() for a in [grid_x, grid_y]], dtype=np.float_) # (2, 25600)
xa = np.asarray([xa for _ in range(0, batch_size)], dtype=np.float_) # (batch_size, 2, 25600)

In [89]:
g = tf.Graph()
with g.as_default():
    # placeholder
    avgP_inputs = tf.placeholder(tf.float32, (None, avgP_num), name='avgP_inputs')
    
    l_x_labels = tf.placeholder(tf.float32, (None, l_num), name='l_x_labels')
    l_y_labels = tf.placeholder(tf.float32, (None, l_num), name='l_y_labels')
    t_labels = tf.placeholder(tf.float32, (None, t_size, t_size, t_channel), name='t_labels')
    w_labels = tf.placeholder(tf.float32, shape=(None, emb_num), name= 'w_labels')
    
    grids = tf.constant(xa, dtype=tf.float32, name= 'grids')
    
    # model
    F = F_layer(avgP_inputs, f_num= f_num)
    
    (l_x_preds, l_y_preds) = landmark_decode(F, landmark_num= l_num)
    
    l_x_loss = tf.losses.mean_squared_error(l_x_labels, l_x_preds, reduction="weighted_mean")
    l_y_loss = tf.losses.mean_squared_error(l_y_labels, l_y_preds, reduction="weighted_mean")
    
    l_loss = tf.add(l_x_loss, l_y_loss)
    
    t_preds = texture_decode(F, t_size)
    t_loss = tf.losses.absolute_difference(t_labels, t_preds)
    
    warp = warp_tf(t_preds, l_x_preds, l_y_preds, l_x_labels, l_y_labels, grids, grid_shape, batch_size)
    cast_warp = tf.cast(warp, tf.uint8)

In [90]:
with g.as_default():
    time_load_data = time.time()
    
    #facenet
    start_load_facenet = time.time()
    print("--- %s start load facenet ---" % (start_load_facenet))
    facenet.load_model('./20171012', input_map={"input:0": warp})
    print("--- %s facenet loaded ---" % (time.time() - start_load_facenet))

    f_phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
    w_preds = tf.get_default_graph().get_tensor_by_name("embeddings:0")    
    w_loss = tf.losses.cosine_distance(tf.nn.l2_normalize(w_labels, 1), 
                                       tf.nn.l2_normalize(w_preds, 1), dim=1)
    
    total_cost = l_loss + t_loss + w_loss
    opt = tf.train.AdamOptimizer(0.001).minimize(total_cost)
    
    variables = [v for v in tf.global_variables()]
    init = tf.variables_initializer(var_list= variables, name= 'init')

--- 1507875598.88635 start load facenet ---
Model filename: ./20171012
--- 3.634227752685547 facenet loaded ---


In [91]:
def show_landmarks(image, landmarks):
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='b')
    plt.pause(0.001)

In [92]:
def show_test(l_x, l_y, t, w, index):
    index = len(train_set) * batch_size + index
    img_name = landmarks_frame.ix[index, 0]
    landmarks = landmarks_frame.ix[n, 1:].as_matrix().astype('float')
    landmarks = landmarks.reshape(-1, 2) # (68, 2)
    
    plt.figure()
    show_landmarks(io.imread(os.path.join('', img_name)), landmarks)
    plt.show()
    
    t_img = scipy.misc.toimage(t)
    plt.imshow(t_img)
    plt.scatter(l_x, l_y, s=10, marker='.', c='b')
    plt.pause(0.001)
    plt.show()
    
#     plt.imshow(np.squeeze(w, axis=0))
#     plt.show()

In [94]:
with g.as_default():
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        print(sess.run(init))
        start_test = time.time()
        
        for e in range(epochs):
            for i, ((f_avgP, f_emb), t_label_batch) in enumerate(zip(train_set, t_dataset.get_batches(batch_size))):
                start = i * batch_size
                end = min(start+batch_size, len(train_set) * batch_size)
                size = end - start

                l_labels = landmarks_frame.ix[start:end - 1, 1:].as_matrix().astype('float').reshape(-1, 2)

                run = [l_x_loss, l_y_loss, t_loss, w_loss, opt]
                feed_dict = {avgP_inputs : f_avgP.reshape(-1, avgP_num),
                             l_x_labels : l_labels[:, 0].reshape(-1, l_num), 
                             l_y_labels : l_labels[:, 1].reshape(-1, l_num),
                             t_labels : t_label_batch, 
                             w_labels : f_emb, 
                             f_phase_train_placeholder:False}

                l_x_cost, l_y_cost, t_cost, w_cost, _ = sess.run(run, feed_dict= feed_dict)
                
                print("Epoch: {}/{}...".format(e+1, epochs),\
                          "Training loss: X = {:.4f}, Y = {:.4f}, T = {:.4f}, W = {:.4f}".format(l_x_cost / size, 
                                                                                                 l_y_cost / size, 
                                                                                                 t_cost, 
                                                                                                 w_cost))
            test_index = random.randint(0, len(test_set[0])-1)
            test_avgP = test_set[0][test_index]
            test_run = [l_x_preds, l_y_preds, t_preds]
            test_feed = {avgP_inputs : test_avgP.reshape(-1, avgP_num)}
            t_l_x, t_l_y, t_t = sess.run(test_run, feed_dict= test_feed)
            show_test(t_l_x, t_l_y, t_t, test_index)
            print("Time: %s" % (time.time() - start_test))

None


KeyboardInterrupt: 

# Embeding Facenet

In [4]:
def load_and_align_data(image_paths, image_size, margin, gpu_memory_fraction):

    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    
    print('Creating networks and loading parameters')
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
  
    nrof_samples = len(image_paths)
    img_list = [None] * nrof_samples
    for i in range(nrof_samples):
        img = misc.imread(os.path.expanduser(image_paths[i]))
        if (img.shape[2] == 4):
            img = img[:, :, :3]
        img_size = np.asarray(img.shape)[0:2]
        bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2, img_size[1])
        bb[3] = np.minimum(det[3]+margin/2, img_size[0])
        cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
        aligned = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
        prewhitened = facenet.prewhiten(aligned)
        img_list[i] = prewhitened
        sys.stdout.write('\r'+ '%d/%d'%(i, nrof_samples))
    images = np.stack(img_list)
    return images

In [8]:
batch_size = 100

f_emb_g = tf.Graph()
f_emb_list = []
f_avgP_list = []

In [9]:
with f_emb_g.as_default():
    time_load_data = time.time()
    print("--- data loading start ---")
    images = load_and_align_data(file_list, 160, 44, 1.0)
    print("--- %s data loaded ---" % (time.time() - time_load_data))
    
    with tf.Session() as sess:
        start_load_facenet = time.time()
        print("--- %s start load facenet ---" % (start_load_facenet))
        facenet.load_model('./20171012')
        print("--- %s facenet loaded ---" % (time.time() - start_load_facenet))

        # Get input and output tensors
        f_inputs = tf.get_default_graph().get_tensor_by_name("input:0")
        f_phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        f_avgP_logits = tf.get_default_graph().get_tensor_by_name("InceptionResnetV1/Logits/AvgPool_1a_8x8/AvgPool:0")
        f_emb_logits = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        
        start_test = time.time()
        
        total_size = len(images)
        for i in range(0, total_size, batch_size):
            batch = images[i:min(i+batch_size, total_size)]
            f_feed = {f_inputs: batch, f_phase_train_placeholder:False}
            f_embedings, f_avgPool = sess.run([f_emb_logits, f_avgP_logits], feed_dict=f_feed)
            f_emb_list.append(f_embedings)
            f_avgP_list.append(f_avgPool)
            sys.stdout.write('\r'+ "[%d/%d] %s" % (i, total_size, time.time() - start_test))
            print()
        print("--- %s seconds ---" % (time.time() - start_test))

--- data loading start ---
Creating networks and loading parameters
7866/7867--- 221.59326314926147 data loaded ---
--- 1507870378.9542003 start load facenet ---
Model filename: ./20171012
--- 4.037825107574463 facenet loaded ---
[0/7867] 4.5784010887146
[100/7867] 5.2621378898620605
[200/7867] 5.945639133453369
[300/7867] 6.6299684047698975
[400/7867] 7.314805746078491
[500/7867] 7.998567819595337
[600/7867] 8.683316469192505
[700/7867] 9.368604898452759
[800/7867] 10.052686929702759
[900/7867] 10.732678651809692
[1000/7867] 11.416769981384277
[1100/7867] 12.100522756576538
[1200/7867] 12.785982131958008
[1300/7867] 13.469927549362183
[1400/7867] 14.15501356124878
[1500/7867] 14.839283466339111
[1600/7867] 15.523703336715698
[1700/7867] 16.209466457366943
[1800/7867] 16.894711017608643
[1900/7867] 17.575763702392578
[2000/7867] 18.26040482521057
[2100/7867] 18.94400978088379
[2200/7867] 19.62877655029297
[2300/7867] 20.31416368484497
[2400/7867] 20.99917984008789
[2500/7867] 21.683804

In [10]:
np.savez('f_emb_list.npz', *f_emb_list)
np.savez('f_avgP_list.npz', *f_avgP_list)